In [10]:
import pandas as pd
import numpy as np
from sklearn.utils import shuffle

### ввод
df = pd.read_csv('./dataset.csv')
df.drop(df.columns[0], axis=1, inplace=True)
df.drop('Post Promotion Status', axis=1, inplace=True)

'''
мб нужен тестовый сет
'''
'''
df= pd.DataFrame()
df['A'] = np.random.randint(0,50000, size=40000)
df['B'] = np.random.randint(0,50, size=40000)
df['C'] = np.random.randint(0,60, size=40000)
df['D'] = np.random.randint(0,75, size=40000)
df['AA'] = np.random.randint(0,5, size=40000)
df['BB'] = np.random.randint(0,15, size=40000)
df['CC'] = np.random.randint(0,3785, size=40000)
df['DD'] = np.random.randint(0,95, size=40000)
df['EE'] = np.random.randint(0,8, size=40000)
df['E'] = np.random.randint(0,115, size=40000)
df['Target'] = np.random.randint(0,100, size=40000)'''



### нормируем
def is_binary(x):
    return x.unique().shape[0] == 2


def normalize(x):
    return (x - x.mean()) / x.std()


bin_free = df.columns[~df.apply(is_binary)]
df[bin_free] = df[bin_free].apply(normalize, axis=0)
df['w0_reg_constant'] = 1

#на всякий вырезал выбросы
df = df.drop(df[df.Target > df.Target.quantile(.95)].index)

### перемешиваем
df = shuffle(df)


# функция градиентного спуска
# где new_w задает измение веса с помощью градиентного спуска
def gradient_descent(X, y, theta=1e-2, epsilon=1e-3):

    w = np.random.normal(size=X.shape[1])
    new_w = 1000*w
    N = X.shape[0]
    iter_num = 0

    while np.linalg.norm(w-new_w) >= epsilon:
        new_w = w
        dw = (2 / N) * (X.T.dot( X.dot (w)) - X.T.dot(y))
        w = w - theta*dw
        
        iter_num+=1
        if iter_num > 1000:
            return w
        
    return w


# метрики и прочее
def R2(x, y):
    return 1 - np.sum(np.power(y - x, 2)) / np.sum(np.power(y - y.mean(), 2))

def reg_prediction(X, w):
    return X.dot( w)

def RMSE(x, y):
    return np.sqrt(np.sum(np.power(y - x, 2)) / y.shape[0])


# делим на фолды и мутим свое грязное дело
# вторые копии для наглядости
folds_index = 5
fold_size = round(df.shape[0] / folds_index)

features = pd.DataFrame()
RMSE_test = []
RMSE_train = []
R2_test = []
R2_train = []

for i in range(folds_index):
    test = df[i * fold_size:(i + 1) * fold_size]
    if i == 0:
        train = df[(i + 1) * fold_size:]
    else:
        train = df[:i * fold_size]
        if i != 4:
            train = train.append(df[(i + 1) * fold_size:], ignore_index=False)

    Features = train.drop('Target', axis=1)
    Target = train['Target']
    w = gradient_descent(Features, Target, 1e-2)
    features = features.append(w, ignore_index=True)
    

    train_pred = reg_prediction(train.drop('Target', axis=1), w)
    R2_train.append(R2(train_pred, train['Target']))
    RMSE_train.append(RMSE(train_pred, train['Target']))

    test_pred = reg_prediction(test.drop('Target', axis=1), w)
    R2_test.append(R2(test_pred, test['Target']))
    RMSE_test.append(RMSE(test_pred, test['Target']))    

# вывод результатов с шагом 0.001
res_df = pd.DataFrame(np.vstack([R2_test, R2_train, RMSE_test, RMSE_train]),
                      index=['R2_test', 'R2_train', 'RMSE_test', 'RMSE_train'])
res_df = res_df.append(features.T)
res_df.columns = ['T1', 'T2', 'T3', 'T4', 'T5']
res_df = pd.concat([res_df, res_df.mean(axis=1).rename('E(mean)'), res_df.std(axis=1).rename('STD')], axis=1)

#print(res_df)
res_df

,T1,T2,T3,T4,T5,E(mean),STD
R2_test,-8.276703,-4.371465,-6.722402,-4.201719,-3.547115,-5.423881,1.996764
R2_train,-8.893991,-5.313985,-4.869528,-4.337598,-3.412508,-5.365522,2.095808
RMSE_test,0.408234,0.321313,0.382521,0.312024,0.291397,0.343098,0.049774
RMSE_train,0.432307,0.342471,0.330812,0.315954,0.287364,0.341782,0.054644
Base Time,-0.042362,-0.060786,-0.053103,-0.049538,-0.038039,-0.048766,0.008944
CC1,0.053268,0.353833,-0.257939,0.183708,1.016177,0.269809,0.473537
CC2,-0.460823,0.559986,-0.387384,0.418967,0.051769,0.036503,0.460315
CC3,0.594310,-0.586983,0.419953,-0.380958,0.241048,0.057474,0.514983
CC4,-0.112532,-0.225535,0.254627,-0.127233,-1.138641,-0.269863,0.518875
CC5,0.723316,-0.681284,0.552674,-0.460703,0.208050,0.068411,0.617437
